<a href="https://colab.research.google.com/github/carolvieirav/Case-Hurb/blob/main/An%C3%A1lisePM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
from geopy import geocoders  
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
from tqdm import tqdm

## Coleta de Dados

In [2]:
df1=pd.read_excel('/content/drive/MyDrive/Desafio Hurb - Metasearch.rar (Unzipped Files)/Base Case Paid Marketing.xlsx', sheet_name=0)
df2=pd.read_excel('/content/drive/MyDrive/Desafio Hurb - Metasearch.rar (Unzipped Files)/Base Case Paid Marketing.xlsx', sheet_name=1)

In [3]:
df1.head()

,Data,Destino,Buscas,Clicks,Custo
0,2019-01-01,Abadiania,13,0,0.000000
1,2019-01-01,Abaete,3,4,0.533333
2,2019-01-01,Abelardo Luz,22,1,0.130000
3,2019-01-01,Aberdeenshire,3,0,0.000000
4,2019-01-01,Abrantes,7,0,0.000000


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915078 entries, 0 to 915077
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   Data     915078 non-null  datetime64[ns]
 1   Destino  915078 non-null  object        
 2   Buscas   915078 non-null  int64         
 3   Clicks   915078 non-null  int64         
 4   Custo    915078 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 34.9+ MB


In [5]:
df2.head()

,Cidade,Data,Pedidos,Faturamento
0,Alto Caparao,2019-01-01,3,1204.245000
1,Chapada dos Guimarães,2019-01-01,1,4041.160000
2,Jericoacoara,2019-01-01,4,21316.266667
3,Punta Del Este,2019-01-01,1,554.260000
4,Marilia,2019-01-01,1,670.300000


In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95944 entries, 0 to 95943
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Cidade       95944 non-null  object        
 1   Data         95944 non-null  datetime64[ns]
 2   Pedidos      95944 non-null  int64         
 3   Faturamento  95944 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 2.9+ MB


In [7]:
df1.nunique()

Data         365
Destino    11966
Buscas     18721
Clicks      1576
Custo      49428
dtype: int64

In [8]:
df2.nunique()

Cidade          2057
Data             365
Pedidos          126
Faturamento    83090
dtype: int64

## Processamento de Dados

In [9]:
df1 = df1.rename(columns={'Destino':'Cidade'}) #mudando o nome da coluna 'destino' para coincidir com a segunda base de dados

In [10]:
df = df1.merge(right=df2, how='outer') # dataframe com as informações das 2 bases de dados 'df1' e 'df2'
df.fillna(0)

,Data,Cidade,Buscas,Clicks,Custo,Pedidos,Faturamento
0,2019-01-01,Abadiania,13.0,0.0,0.000000,0.0,0.000
1,2019-01-01,Abaete,3.0,4.0,0.533333,0.0,0.000
2,2019-01-01,Abelardo Luz,22.0,1.0,0.130000,0.0,0.000
3,2019-01-01,Aberdeenshire,3.0,0.0,0.000000,0.0,0.000
4,2019-01-01,Abrantes,7.0,0.0,0.000000,0.0,0.000
...,...,...,...,...,...,...,...
916460,2019-12-30,Formosa,0.0,0.0,0.000000,3.0,4834.755
916461,2019-12-31,Mayor Drummond,0.0,0.0,0.000000,1.0,3009.270
916462,2019-12-31,Jaguaripe,0.0,0.0,0.000000,1.0,1317.680
916463,2019-12-31,Analandia,0.0,0.0,0.000000,1.0,249.000


In [39]:
# convertendo euro para real com a cotação do dia 29/09/2021
df['Custo'] = df['Custo'].apply(lambda x: x * 6,30)
df['Custo'] = df['Custo'].notnull()

In [40]:
df['Custo'].isnull().sum()

0

In [11]:
df['Roi'] = (df['Faturamento'] - df['Custo']) / df['Custo'] #calculando o Roi de todas as compras relizadas
df.fillna(0)

,Data,Cidade,Buscas,Clicks,Custo,Pedidos,Faturamento,Roi
0,2019-01-01,Abadiania,13.0,0.0,0.000000,0.0,0.000,0.0
1,2019-01-01,Abaete,3.0,4.0,0.533333,0.0,0.000,0.0
2,2019-01-01,Abelardo Luz,22.0,1.0,0.130000,0.0,0.000,0.0
3,2019-01-01,Aberdeenshire,3.0,0.0,0.000000,0.0,0.000,0.0
4,2019-01-01,Abrantes,7.0,0.0,0.000000,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...
916460,2019-12-30,Formosa,0.0,0.0,0.000000,3.0,4834.755,0.0
916461,2019-12-31,Mayor Drummond,0.0,0.0,0.000000,1.0,3009.270,0.0
916462,2019-12-31,Jaguaripe,0.0,0.0,0.000000,1.0,1317.680,0.0
916463,2019-12-31,Analandia,0.0,0.0,0.000000,1.0,249.000,0.0


In [18]:
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [ ]:
tqdm.pandas()
df['location'] = df['Cidade'].progress_apply(geocode)

In [ ]:
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [44]:
df.to_excel('base de dados.xlsx')